In [1]:
!pip install -U nnsight -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.1 MB/s eta 0:00:00


In [2]:
from IPython.display import clear_output
import einops
import torch
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"

import nnsight
from nnsight import LanguageModel, util

In [3]:
from getpass import getpass
from google.colab import userdata
import gc
import weakref

In [4]:
github_token = userdata.get("GITHUB_TOKEN")

# Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/TinySQL.git

import TinySQL as qts

  Cloning https://****@github.com/withmartian/TinySQL.git to /tmp/pip-req-build-jo7clth0
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/withmartian/TinySQL.git' /tmp/pip-req-build-jo7clth0
  Resolved https://****@github.com/withmartian/TinySQL.git to commit f0a39572e84baa3cc434f83d245c68e1ef3da0b8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TinySQL: filename=TinySQL-1.0-py3-none-any.whl size=51800 sha256=f232fdaf52f4600d494b795e4438e518173ada018c72cc9238990cdc330a6e10
  Stored in directory: /tmp/pip-ephem-wheel-cache-fgsrh0l3/wheels/a9/72/1c/ba39c57a9acd2e1efd9554f1a2ed8d8987923fbad44ab80971
Successfully built TinySQL


In [8]:
! pip install transformer_lens jaxtyping -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


- generate a dataset with clean prompt, corrupted prompt and tokens of the word corrupted
- run the EAP as per the base EAP notebook
- run everything with nnsight


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer

base_model_name = "roneneldan/TinyStories-2Layers-33M"
model_name = "withmartian/sql_interp_bm1_cs1_experiment_1.10"
hf_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer_sql = AutoTokenizer.from_pretrained(base_model_name)

def modify_hf_wte_copy(model: AutoModelForCausalLM) -> AutoModelForCausalLM:

    config = model.config
    new_model = AutoModelForCausalLM.from_config(config)
    new_model.load_state_dict(model.state_dict(), strict=False)

    original_wte = model.transformer.wte
    original_weight = original_wte.weight.data  # Shape: [vocab_size, hidden_size]

    new_weight = original_weight[:-1, :]  # Shape: [vocab_size - 1, hidden_size]

    new_wte = torch.nn.Embedding(new_weight.size(0), new_weight.size(1))
    new_wte.weight.data = new_weight

    new_model.transformer.wte = new_wte

    original_unembed_weight = model.lm_head.weight.data  # Shape: [vocab_size, hidden_size]

    new_unembed_weight = original_unembed_weight[:-1, :]  # Shape: [vocab_size - 1, hidden_size]

    new_model.lm_head = torch.nn.Linear(new_unembed_weight.size(1), new_unembed_weight.size(0), bias=False)
    new_model.lm_head.weight.data = new_unembed_weight

    return new_model

fixed_hf_model = modify_hf_wte_copy(hf_model)

model_sql = HookedTransformer.from_pretrained(
            "roneneldan/TinyStories-2Layers-33M",
            hf_model=fixed_hf_model,
            device= 'cuda',
            fold_ln=False,
            center_writing_weights=False,
            center_unembed=False,
            tokenizer=tokenizer_sql,
        )

model_sql.cfg.use_attn_result = True
model_sql.cfg.attn_only = False
model_sql.cfg.use_hook_mlp_in = True
model_sql.cfg.use_split_qkv_input = True

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Loaded pretrained model roneneldan/TinyStories-2Layers-33M into HookedTransformer


In [130]:
model_num = 1                     # 0=GPT2, 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 1                        # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3
feature_name = qts.DEFFIELDNAME   # ENGTABLENAME, ENGFIELDNAME, DEFTABLESTART, DEFTABLENAME, DEFFIELDNAME, DEFFIELDSEPARATOR
use_novel_names = False           # If True, we corrupt using words not found in the clean prompt or create sql e.g. "little" or "hammer"

In [131]:
# Key global "input" variables
clean_prompt = ""
corrupt_prompt = ""
clean_tokenizer_index = qts.UNKNOWN_VALUE # Tokenizer vocab index for clean word
corrupt_tokenizer_index = qts.UNKNOWN_VALUE # Tokenizer vocab index for corrupted word
answer_token_index = qts.UNKNOWN_VALUE # Token index in sql command answer of clean/corrupt word

# Key global "results" variables
clean_logit_diff = qts.UNKNOWN_VALUE
corrupt_logit_diff = qts.UNKNOWN_VALUE

In [132]:
if model_num > 0:
    # Generate a batch of clean and corrupt prompts for feature_name
    batch_size = 30
    generator = qts.CorruptFeatureTestGenerator(model_num, cs_num, tokenizer_sql, use_novel_names=use_novel_names)
    examples = generator.generate_feature_examples(feature_name, batch_size)

examples_prep = []
for example in examples:

    clean_tokenizer_index = example.clean_tokenizer_index
    corrupt_tokenizer_index = example.corrupt_tokenizer_index
    answer_token_index = example.answer_token_index

    # Truncate the clean_prompt at answer_token_index
    clean_prompt = example.clean_BatchItem.get_alpaca_prompt() + example.clean_BatchItem.sql_statement
    clean_tokens = tokenizer_sql(clean_prompt)["input_ids"]
    clean_tokens = clean_tokens[:answer_token_index]
    clean_prompt = tokenizer_sql.decode(clean_tokens)

    # Truncate the corrupt_prompt at answer_token_index
    corrupt_prompt = example.corrupt_BatchItem.get_alpaca_prompt() + example.corrupt_BatchItem.sql_statement
    corrupt_tokens = tokenizer_sql(corrupt_prompt)["input_ids"]
    corrupt_tokens = corrupt_tokens[:answer_token_index]
    corrupt_prompt = tokenizer_sql.decode(corrupt_tokens)

    examples_prep.append({
        "clean_prompt": clean_prompt,
        "clean_tokens": clean_tokens,
        "corrupt_prompt": corrupt_prompt,
        "corrupt_tokens": corrupt_tokens,
        "clean_tokenizer_index": example.clean_tokenizer_index,
        "corrupt_tokenizer_index": example.corrupt_tokenizer_index,
    })


In [133]:
clean_prompt, corrupt_prompt

('### Instruction: show me the amount and total from the inventory table ### Context: CREATE TABLE inventory ( amount TIME, total TEXT ) ### Response: SELECT',
 '### Instruction: show me the amount and total from the inventory table ### Context: CREATE TABLE inventory ( id TIME, total TEXT ) ### Response: SELECT')

In [16]:
!git clone -b minimal-implementation --single-branch https://github.com/Aaquib111/edge-attribution-patching.git


Cloning into 'edge-attribution-patching'...
remote: Enumerating objects: 7461, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 7461 (delta 86), reused 88 (delta 67), pack-reused 7326 (from 1)
Receiving objects: 100% (7461/7461), 1.58 GiB | 17.63 MiB/s, done.
Resolving deltas: 100% (3719/3719), done.


In [105]:
%cd /content/edge-attribution-patching

from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.magic("%load_ext autoreload")
    ipython.magic("%autoreload 2")

import torch

import torch as t
from torch import Tensor
import einops

from transformer_lens import HookedTransformer

from eap.eap_wrapper import EAP

from jaxtyping import Float

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/content/edge-attribution-patching
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Device: cuda


In [134]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    examples_prep,
    per_prompt: bool = False
):
    '''
    Return average logit difference between correct and incorrect answers
    '''
    # Access .logits if it's a CausalLMOutputWithPast object
    if hasattr(logits, 'logits'):
        logits = logits.logits

    batch_size = logits.size(0)

    # Get logits for clean and corrupt tokens at the last position (-1)
    clean_logits = logits[range(batch_size), -1, [example["clean_tokenizer_index"] for example in examples_prep]]
    corrupt_logits = logits[range(batch_size), -1, [example["corrupt_tokenizer_index"] for example in examples_prep]]

    logit_diff = clean_logits - corrupt_logits
    return logit_diff if per_prompt else logit_diff.mean()

# Get clean and corrupt logit differences
with t.no_grad():
    clean_logits = model_sql(t.tensor([example["clean_tokens"] for example in examples_prep]).to('cuda'))
    corrupt_logits = model_sql(t.tensor([example["corrupt_tokens"] for example in examples_prep]).to('cuda'))
    clean_logit_diff = ave_logit_diff(clean_logits, examples_prep).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, examples_prep).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    examples = examples_prep
):
    patched_logit_diff = ave_logit_diff(logits, examples)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)

# Get clean and corrupt metrics
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, examples_prep)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, examples_prep)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 7.21671199798584, Corrupt direction: 5.214576721191406
Clean metric: 1.0, Corrupt metric: 0.0


In [135]:
# Prepare inputs
clean_tokens = t.tensor([example["clean_tokens"] for example in examples_prep]).to('cuda')
corrupt_tokens = t.tensor([example["corrupt_tokens"] for example in examples_prep]).to('cuda')

# Run EAP
graph = EAP(
    model_sql,
    clean_tokens,
    corrupt_tokens,
    ioi_metric,
    upstream_nodes=["mlp", "head"],
    downstream_nodes=["mlp", "head"]
)
top_edges = graph.top_edges(n=10, abs_scores=True)
for from_edge, to_edge, score in top_edges:
    print(f'{from_edge} -> [{round(score, 3)}] -> {to_edge}')

Saving activations requires 0.0001 GB of memory per token


100%|██████████| 30/30 [00:06<00:00,  4.84it/s]


mlp.0 -> [-0.079] -> head.1.8.k
mlp.0 -> [0.041] -> head.1.3.v
head.1.3 -> [0.035] -> mlp.1
head.0.8 -> [0.023] -> mlp.0
mlp.0 -> [-0.018] -> head.1.7.v
head.0.14 -> [0.018] -> mlp.0
mlp.0 -> [0.017] -> head.1.8.v
head.1.7 -> [-0.017] -> mlp.1
head.1.8 -> [0.017] -> mlp.1
head.0.3 -> [0.015] -> mlp.0


In [128]:
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict
from typing import Dict, List, Set, Tuple, Optional

def plot_eap_sankey(top_edges, score_threshold=1e-2, layer_spacing=False):
    # Get unique nodes
    nodes = set()
    for from_edge, to_edge, _ in top_edges:
        nodes.add(from_edge)
        nodes.add(to_edge)
    nodes = list(nodes)
    node_idx = {node: idx for idx, node in enumerate(nodes)}

    # Color palette for nodes
    COLOR_PALETTE = [
        "rgba(31, 119, 180, 0.8)",    # blue
        "rgba(255, 127, 14, 0.8)",    # orange
        "rgba(44, 160, 44, 0.8)",     # green
        "rgba(214, 39, 40, 0.8)",     # red
        "rgba(148, 103, 189, 0.8)",   # purple
        "rgba(140, 86, 75, 0.8)",     # brown
        "rgba(227, 119, 194, 0.8)",   # pink
        "rgba(127, 127, 127, 0.8)",   # gray
        "rgba(188, 189, 34, 0.8)",    # yellow
        "rgba(23, 190, 207, 0.8)"     # cyan
    ]

    # Assign unique colors to nodes
    node_colors = {}
    for i, node in enumerate(nodes):
        node_colors[node] = COLOR_PALETTE[i % len(COLOR_PALETTE)]

    # Define the sankey nodes
    node_labels = [node for node in nodes]
    node_colors = [node_colors[node] for node in nodes]

    # Setup for layer spacing
    lyr_nodes: Dict[int, List[str]] = defaultdict(list)
    for node in nodes:
        layer = int(node.split('.')[1]) if '.' in node else 0
        lyr_nodes[layer].append(node)

    # Define the sankey edges
    sources, targets, values, labels, colors = [], [], [], [], []
    included_layer_nodes: Dict[int, List[str]] = defaultdict(list)

    for from_edge, to_edge, score in top_edges:
        if abs(score) < score_threshold:
            continue

        source_idx = node_idx[from_edge]
        target_idx = node_idx[to_edge]

        sources.append(source_idx)
        targets.append(target_idx)
        values.append(abs(score))
        labels.append(f"{from_edge} → {to_edge}<br>{score:.3f}")

        if score == 0:
            edge_color = "rgba(0,0,0,0.1)"
        elif score > 0:
            edge_color = "rgba(0,0,255,0.3)"
        else:
            edge_color = "rgba(255,0,0,0.3)"
        colors.append(edge_color)

        source_layer = int(from_edge.split('.')[1]) if '.' in from_edge else 0
        target_layer = int(to_edge.split('.')[1]) if '.' in to_edge else 0
        included_layer_nodes[source_layer].append(from_edge)
        included_layer_nodes[target_layer].append(to_edge)

    if layer_spacing:
        ordered_lyr_nodes = [nodes for _, nodes in sorted(included_layer_nodes.items())]
        ghost_edge_val = 1e-6

        for lyr_1_nodes, lyr_2_nodes in zip(ordered_lyr_nodes[:-1], ordered_lyr_nodes[1:]):
            first_lyr_1_node = lyr_1_nodes[0]
            first_lyr_2_node = lyr_2_nodes[0]

            for lyr_1_node in lyr_1_nodes:
                sources.append(node_idx[lyr_1_node])
                targets.append(node_idx[first_lyr_2_node])
                values.append(ghost_edge_val)
                labels.append("")
                colors.append("rgba(0,255,0,0.0)")

            for lyr_2_node in lyr_2_nodes:
                sources.append(node_idx[first_lyr_1_node])
                targets.append(node_idx[lyr_2_node])
                values.append(ghost_edge_val)
                labels.append("")
                colors.append("rgba(0,255,0,0.0)")

    # Create the Sankey diagram
    fig = go.Figure(go.Sankey(
        arrangement="perpendicular",
        node=dict(
            label=node_labels,
            color=node_colors,
            line=dict(width=0.0),
            pad=15,
            thickness=20,
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            label=labels,
            color=colors,
            arrowlen=25
        ),
        domain={'y': [0, 1]}
    ))

    # Update layout
    n_layers = len(included_layer_nodes)
    h = max(250, 400)
    w = max(50 * n_layers, 600)

    fig.update_layout(
        font_size=14,
        height=h,
        width=w,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=20, l=20, r=20, b=20),
        hoverlabel=dict(font_size=14)
    )

    return fig

In [138]:
fig = plot_eap_sankey(top_edges, score_threshold=1e-2)
fig.show()